# Computer Vision push ups counter

In this project I have combined two techniques to construct a simple computer vision technique to count the number of push ups, pull ups or squats from a video.

## Creating the validation and train data generators, from the directories. 

In [1]:
# import the necessary packages

import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import distutils


# initialize ImageDataGenerator for training data because the amount of data is not enough
train_datagen = ImageDataGenerator(
    rescale=1./255,                     # nomalize the image
    shear_range=0.2,                    # apply random shearing to images
    zoom_range=0.2,                     # apply random zoom in/out to images
    horizontal_flip=True)               # flip the image horizontally

test_datagen = ImageDataGenerator(rescale=1./255)    # because the training data is normalized, we also need to normalize the test data


train_generator = train_datagen.flow_from_directory(
    './newdata/train',
    target_size=(64, 64),               # resize input images to 64x64 pixels
    color_mode="rgb",                   # use RGB color space for images for esier to train 
    batch_size=16,                      # use batch size of 16  
    class_mode='categorical',           # use categorical crossentropy loss function
    shuffle=True,                       # shuffle the training data
    seed=42)                            # set random seed to standard value for reproducibility

# same as above
validation_generator = test_datagen.flow_from_directory(
    './newdata/validation',
    target_size=(64, 64),               
    batch_size=1,
    class_mode='categorical')



Found 734 images belonging to 3 classes.
Found 117 images belonging to 3 classes.


## Model

In [15]:
def create_model():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.Input(shape=(64,64,3)))

  model.add(tf.keras.layers.Conv2D(6, (3, 3), padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  
  model.add(tf.keras.layers.Conv2D(16, (3,3), padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2))) 

  model.add(tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

  model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))  

  model.add(tf.keras.layers.Dropout(0.25))                 # prevent overfitting

  model.add(tf.keras.layers.Flatten())                     # convert 3D feature maps to 1D feature vectors
  model.add(tf.keras.layers.Dense(512)) 
  model.add(tf.keras.layers.Activation('relu'))
  model.add(tf.keras.layers.Dropout(0.5)) 
  model.add(tf.keras.layers.Dense(128))
  model.add(tf.keras.layers.Activation('relu'))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(64))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(3))                       # 3 classes : no_move, up_move, down_move
  model.add(tf.keras.layers.Activation('softmax'))
  return model

In [31]:
# structure of the model
model = create_model()
model.summary()


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_44 (Conv2D)          (None, 64, 64, 6)         168       
                                                                 
 max_pooling2d_44 (MaxPooli  (None, 32, 32, 6)         0         
 ng2D)                                                           
                                                                 
 conv2d_45 (Conv2D)          (None, 32, 32, 16)        880       
                                                                 
 max_pooling2d_45 (MaxPooli  (None, 16, 16, 16)        0         
 ng2D)                                                           
                                                                 
 conv2d_46 (Conv2D)          (None, 16, 16, 32)        4640      
                                                                 
 max_pooling2d_46 (MaxPooli  (None, 8, 8, 32)        

## Initializing the model and training

In [5]:

# model = load_model('model.h5')   #use this line of code to continues train the model if it already exists 
model = create_model()
model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),        # use Adam for better performance
      loss='categorical_crossentropy',                               # use categorical_crossentropy loss function                              
      metrics=['categorical_accuracy'])                              # optimze the model based on categorical_accuracy

model.fit(
    train_generator,
    epochs=20,                                        # train the model for 20 epochs                           
    validation_data=validation_generator,             
    validation_freq=1                                 # calculate validation accuracy after each epoch    
)

# save it to use or train later 
model.save('model.h5', overwrite=True) 

Epoch 1/20
46/46 [==============================] - 11s 211ms/step - loss: 0.6588 - categorical_accuracy: 0.6907 - val_loss: 0.3611 - val_categorical_accuracy: 0.8889
Epoch 2/20
46/46 [==============================] - 9s 200ms/step - loss: 0.2765 - categorical_accuracy: 0.8978 - val_loss: 0.3575 - val_categorical_accuracy: 0.8718
Epoch 3/20
46/46 [==============================] - 9s 200ms/step - loss: 0.2447 - categorical_accuracy: 0.9196 - val_loss: 0.2100 - val_categorical_accuracy: 0.9487
Epoch 4/20
46/46 [==============================] - 9s 198ms/step - loss: 0.2509 - categorical_accuracy: 0.9428 - val_loss: 0.1696 - val_categorical_accuracy: 0.9487
Epoch 5/20
46/46 [==============================] - 9s 202ms/step - loss: 0.2561 - categorical_accuracy: 0.9114 - val_loss: 0.2460 - val_categorical_accuracy: 0.9316
Epoch 6/20
46/46 [==============================] - 9s 200ms/step - loss: 0.1924 - categorical_accuracy: 0.9605 - val_loss: 0.2113 - val_categorical_accuracy: 0.9573
Epo

c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Loading the model 

In [9]:

model = tf.keras.models.load_model('model.h5')

## Loading and generating a new video with the repetitions

In [10]:
import cv2
import numpy as np
cap = cv2.VideoCapture("3.mp4")

fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
out = cv2.VideoWriter('output_count.avi',fourcc, 20.0,(int(cap.get(3)),int(cap.get(4))))

ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255
i= 0
prediction_str = ""
repetitions = 0
up = 0
down = 0
no_move = 0
current_move = 0
initial = -1
while(cap.isOpened()):
    i+=1
    
    ret, frame2 = cap.read()
    if not(ret): break
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    

    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)

    image = cv2.resize(rgb, (64, 64))
    image = image.reshape((1,) + image.shape)
    image = image/255.0
    prediction = np.argmax(model.predict(image), axis=-1)[0]
    
    if prediction == 0:
        down +=1 
        if down == 3:
          if initial == -1:
            initial = 0
          if current_move == 2:
            repetitions+=1
          current_move = 0
        elif down > 0:
          up = 0
          no_move = 0
    elif prediction == 2:
        up += 1
        if up == 3 and initial != -1:
          current_move = 2
        elif up > 1:
          down = 0 
          no_move = 0
    else:
        no_move += 1
        if no_move == 15:
          current_move = 1
        elif no_move > 10:
          up = 0
          down = 0 
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (10,400)
    fontScale              = 1
    fontColor              = (255,255,255)
    lineType               = 5
    cv2.putText(frame2, "Repetitions: "+ str(repetitions),bottomLeftCornerOfText,font, fontScale,fontColor,lineType)
    out.write(frame2)
    prvs = next

print("Video Generated")
out.release()
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 21ms/step
Video Generated
